# Understanding the structure of cities through the lens of data

#### SDSS 2022 Workshop by:

### [Martin Fleischmann](https://martinfleischmann.net)<sup>1,2,3</sup> & [James D. Gaboardi](https://www.ornl.gov/staff-profile/james-d-gaboardi)<sup>4</sup>
  
1. Geographic Data Science Lab, University of Liverpool
2. Urban and Regional Laboratory, Charles University
3. UrbanDataLab AG, Zurich
4. Oak Ridge National Laboratory

<hr>

Materials:

<div style="font-size:40px;">
github.com/martinfleis/sdss22-workshop
</div>
<br>

Open in an interactive in-browser environment: 

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/martinfleis/sdss22-workshop/HEAD?labpath=demo-notebook.ipynb)

In [1]:
%%html
 <iframe height="750" width="100%" src="https://martinfleis.github.io/sdss22-workshop/slides/intro.html", style="border:0"></iframe>

## Agenda
1. Wrangling input data
2. Generating & combining morphometric elements
3. Measuring morphometric characters
4. Understanding context & clustering
5. Wrap up
-------------------------

In [ ]:
%config InlineBackend.figure_format = "retina"
%load_ext watermark
%watermark

In [ ]:
import bokeh
from bokeh.io import output_notebook
from bokeh.plotting import show
import clustergram
import contextily
import geopandas
import libpysal
import matplotlib.pyplot as plt
import momepy
import osmnx
import pandas
import tqdm
import warnings
%watermark -iv
bokeh.io.output_notebook()

----------------------------------

## 1. Wrangling input data

### Pick a place and set its local CRS.
This should ideally be a town with a good coverage in [OpenStreetMap](https://www.openstreetmap.org).

In [ ]:
place = "St. Gallen, St. Gallen, Schweiz"
local_crs = 2056

Explore with [`geopandas`](https://geopandas.org/en/stable/).

In [ ]:
geopandas.tools.geocode(place).explore()

### Download data from OpenStreetMap.

Query OSM with [`osmnx`](https://osmnx.readthedocs.io/en/stable/).

#### Buildings

In [ ]:
buildings = osmnx.geometries.geometries_from_place(place, tags={"building": True})
buildings.head()

In [ ]:
buildings.geom_type.value_counts()

In [ ]:
buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)

In [ ]:
buildings.explore(prefer_canvas=True, tiles="CartoDB Positron", tooltip=False)

In [ ]:
buildings = buildings[["geometry"]].to_crs(local_crs)

In [ ]:
buildings["uID"] = range(len(buildings))
buildings

#### Streets

In [ ]:
osm_graph = osmnx.graph_from_place(place, network_type="drive")

In [ ]:
osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)

In [ ]:
streets = osmnx.graph_to_gdfs(osm_graph, nodes=False, node_geometry=False)
streets.head()

In [ ]:
streets.explore(prefer_canvas=True, tiles="CartoDB Positron")

In [ ]:
streets = momepy.remove_false_nodes(streets)
streets = streets[["geometry"]]
streets["nID"] = range(len(streets))
streets

---------------

## 2. Generating & combining morphometric elements
### Create a tessellation.

Given the following building footprints:

![blg](http://docs.momepy.org/en/stable/_images/user_guide_elements_tessellation_3_0.png)

We can generate a spatial units using Voronoi tessellation:

In [ ]:
bubenec = momepy.datasets.get_path("bubenec")
bubenec_blg = geopandas.read_file(bubenec, layer="buildings")
bubenec_tess = geopandas.read_file(bubenec, layer="tessellation")

In [ ]:
toner_lite = contextily.providers.Stamen.TonerLite

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    ax = bubenec_blg.plot(figsize=(16, 16))
    bubenec_tess.plot(ax=ax, facecolor="none", edgecolor="r")
    contextily.add_basemap(ax=ax, crs=bubenec_tess.crs, source=toner_lite)
    ax.set_axis_off()

In [ ]:
limit = momepy.buffered_limit(buildings, 100)
tessellation = momepy.Tessellation(buildings, "uID", limit, segment=1).tessellation

#### Link streets

Link unique IDs of streets to buildings and tessellation cells based on the nearest neighbor join.

In [ ]:
buildings = buildings.sjoin_nearest(streets, max_distance=1000, how="left")
buildings

In [ ]:
buildings = buildings.drop_duplicates("uID").drop(columns="index_right")

In [ ]:
tessellation = tessellation.merge(buildings[["uID", "nID"]], on="uID", how="left")

--------------------


## 3. Measuring morphometric characters
### Measure individual morphometric characters.
#### Dimensions

In [ ]:
buildings["area"] = buildings.area
tessellation["area"] = tessellation.area
streets["length"] = streets.length

#### Shape

In [ ]:
buildings["eri"] = momepy.EquivalentRectangularIndex(buildings).series

In [ ]:
buildings["elongation"] = momepy.Elongation(buildings).series

In [ ]:
tessellation["convexity"] = momepy.Convexity(tessellation).series

In [ ]:
streets["linearity"] = momepy.Linearity(streets).series

Declare some plotting symbology and cartographic elements that we can reuse.

In [ ]:
symb_kws = dict(scheme="natural_breaks", legend=True)
font_kws = dict(fontdict={"fontsize":20})

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))
for i, col in enumerate(["eri", "elongation"]):
    buildings.plot(col, ax=ax[i], **symb_kws)
    ax[i].set_title(col, **font_kws)
    ax[i].set_axis_off()
fig.subplots_adjust(wspace=-.05)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))
for i, (df, col) in enumerate(zip([tessellation, streets], ["convexity", "linearity"])):
    df.plot(col, ax=ax[i], **symb_kws)
    ax[i].set_title(col, **font_kws)
    ax[i].set_axis_off()
fig.subplots_adjust(wspace=-.05)

#### Spatial distribution

In [ ]:
buildings["shared_walls"] = momepy.SharedWallsRatio(buildings).series

In [ ]:
ax = buildings.plot("shared_walls", figsize=(12, 6), **symb_kws)
ax.set_title("shared_walls", **font_kws)
ax.set_axis_off()

Generate spatial weights matrices using [`libpysal`](https://github.com/pysal/libpysal).

What are they?

In [ ]:
w = libpysal.weights.Queen.from_dataframe(bubenec_tess)

In [ ]:
edge_kws = dict(color="b", linewidth=1)
node_kws = dict(marker="")
wght_kws = dict(edge_kws=edge_kws, node_kws=node_kws)

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    ax = bubenec_blg.plot(figsize=(16, 16), color="lightgray")
    bubenec_tess.plot(ax=ax, facecolor="none", edgecolor="r")
    w.plot(bubenec_blg, ax=ax, **wght_kws)
    contextily.add_basemap(ax=ax, crs=bubenec_tess.crs, source=toner_lite)
    ax.set_axis_off()

In [ ]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(
    tessellation, ids="uID", silence_warnings=True
)

In [ ]:
tessellation["neighbors"] = momepy.Neighbors(
    tessellation, queen_1, "uID", weighted=True
).series

In [ ]:
tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_1, "uID").series

In [ ]:
buildings["neighbor_distance"] = momepy.NeighborDistance(
    buildings, queen_1, "uID"
).series

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))
for i, (df, col) in enumerate(
    zip([tessellation, buildings], ["covered_area", "neighbor_distance"])
):
    df.plot(col, ax=ax[i], **symb_kws)
    ax[i].set_title(col, **font_kws)
    ax[i].set_axis_off()
fig.subplots_adjust(wspace=-.05)

In [ ]:
queen_3 = libpysal.weights.higher_order(queen_1, k=3, lower_order=True, silence_warnings=True)
buildings_q1 = libpysal.weights.contiguity.Queen.from_dataframe(
    buildings, silence_warnings=True
)
buildings["interbuilding_distance"] = momepy.MeanInterbuildingDistance(
    buildings, queen_1, "uID", queen_3
).series
buildings["adjacency"] = momepy.BuildingAdjacency(
    buildings, queen_3, "uID", buildings_q1
).series

Visualizing actual shared-wall building adjacency:

In [ ]:
buildings_sample = libpysal.weights.contiguity.Queen.from_dataframe(
    bubenec_blg, silence_warnings=True
)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    ax = bubenec_blg.plot(figsize=(16, 16), color="lightgray")
    buildings_sample.plot(bubenec_blg, ax=ax, **wght_kws)
    bubenec_blg.centroid.plot(ax=ax, color='black')
    contextily.add_basemap(ax=ax, crs=bubenec_tess.crs, source=toner_lite)
    ax.set_axis_off()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))
for i, col in enumerate(["interbuilding_distance", "adjacency"]):
    buildings.plot(col, ax=ax[i], **symb_kws)
    ax[i].set_title(col, **font_kws)
    ax[i].set_axis_off()
fig.subplots_adjust(wspace=-.05)

In [ ]:
profile = momepy.StreetProfile(streets, buildings)

In [ ]:
streets["width"] = profile.w

In [ ]:
streets["width_deviation"] = profile.wd

In [ ]:
streets["openness"] = profile.o

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(24, 12))
for i, col in enumerate(["width", "width_deviation", "openness"]):
    streets.plot(col, ax=ax[i], **symb_kws)
    ax[i].set_title(col, **font_kws)
    ax[i].set_axis_off()
fig.subplots_adjust(wspace=-.05)

#### Intensity

In [ ]:
tessellation["car"] = momepy.AreaRatio(
    tessellation, buildings, "area", "area", "uID"
).series

In [ ]:
ax = tessellation.plot("car", figsize=(12, 6), vmin=0, vmax=1, legend=True)
ax.set_title("car", **font_kws)
ax.set_axis_off()

#### Connectivity

In [ ]:
graph = momepy.gdf_to_nx(streets)

In [ ]:
graph = momepy.node_degree(graph)
graph = momepy.closeness_centrality(graph, radius=400, distance="mm_len")
graph = momepy.meshedness(graph, radius=400, distance="mm_len")

In [ ]:
nodes, streets = momepy.nx_to_gdf(graph)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(24, 12))
for i, col in enumerate(["degree", "closeness", "meshedness"]):
    if col == "closeness":
        lgnd_kwds = {"fmt": "{:.6f}"}
        nodes.plot(col, ax=ax[i], markersize=1, legend_kwds=lgnd_kwds, **symb_kws)
    else:
        nodes.plot(col, ax=ax[i], markersize=1, **symb_kws)
    ax[i].set_title(col, **font_kws)
    ax[i].set_axis_off()
fig.subplots_adjust(wspace=-.05)

In [ ]:
buildings["nodeID"] = momepy.get_node_id(buildings, nodes, streets, "nodeID", "nID")

Link all data together (to tessellation cells or buildings).

In [ ]:
tessellation

In [ ]:
merged = tessellation.merge(buildings.drop(columns=["nID", "geometry"]), on="uID")
merged = merged.merge(streets.drop(columns="geometry"), on="nID", how="left")
merged = merged.merge(nodes.drop(columns="geometry"), on="nodeID", how="left")
merged.columns

## 4. Understanding context & clustering

### Context
Measure first, second, and third quartile of distribution of values within an area around each building.

In [ ]:
drop_cols = ["uID", "nodeID", "nID", "mm_len", "node_start", "node_end", "geometry"]

In [ ]:
percentiles = []
for column in tqdm.auto.tqdm(merged.columns.drop(drop_cols)):
    perc = momepy.Percentiles(merged, column, queen_3, "uID", verbose=False).frame
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)

In [ ]:
percentiles_joined = pandas.concat(percentiles, axis=1)
percentiles_joined

In [ ]:
merged["convexity_50"] = percentiles_joined["convexity_50"].values

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))
for i, (df, col) in enumerate(
    zip([tessellation, merged], ["convexity", "convexity_50"])
):
    df.plot(col, ax=ax[i], **symb_kws)
    ax[i].set_title(col, **font_kws)
    ax[i].set_axis_off()
fig.subplots_adjust(wspace=-.05)

### Clustering

Standardize values before clustering with [`clustergram`](https://github.com/martinfleis/clustergram).

In [ ]:
standardized = (
    percentiles_joined - percentiles_joined.mean()
) / percentiles_joined.std()
standardized

#### How many clusters?

In [ ]:
cgram = clustergram.Clustergram(range(1, 12), n_init=10, random_state=0)
cgram.fit(standardized.fillna(0))

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    show(cgram.bokeh())

In [ ]:
cgram.labels.head()

In [ ]:
merged["cluster"] = cgram.labels[8].values

In [ ]:
urban_types = buildings[["geometry", "uID"]].merge(merged[["uID", "cluster"]], on="uID")

In [ ]:
urban_types.explore(
    "cluster", categorical=True, prefer_canvas=True, tiles="CartoDB Positron",
)

-------------------------------------------

# Thank you!

--------------------------------------------------

## 5. Wrap up

### What now?

- `momepy` documentation ([docs.momepy.org](https://docs.momepy.org))
- real-life method to detect urban types ([github.com/martinfleis/numerical-taxonomy-paper](https://github.com/martinfleis/numerical-taxonomy-paper))
- get in touch!
| | Martin | James |
| -- | ------- | ------- | 
| **email:** | martin@martinfleischmann.net | gaboardijd@ornl.gov
| **github:** | [@martinfleis](https://github.com/martinfleis) | [@jGaboardi](https://github.com/jGaboardi)

## References

* **Alessandro Araldi and Giovanni Fusco**. (2019) From the street to the metropolitan region: Pedestrian perspective in urban fabric analysis. *Environment and Planning B: Urban Analytics and City Science*. 46(7):1243–1263. doi: [10.1177/2399808319832612](https://doi.org/10.1177/2399808319832612).
* **Melih Basaraner and Sinan Cetinkaya**. (2017) Performance of shape indices and classification schemes for characterising perceptual shape complexity of building footprints in GIS. *International Journal of Geographical Information Science*. 31(10):1952–1977. doi: [10.1080/13658816.2017.1346257](https://doi.org/10.1080/13658816.2017.1346257).
* **Geoff Boeing.** (2017) OSMnx: New Methods for Acquiring, Constructing, Analyzing, and Visualizing Complex Street Networks. *Computers, Environment and Urban Systems* 65:126-139. doi: [10.1016/j.compenvurbsys.2017.05.004](https://doi.org/10.1016/j.compenvurbsys.2017.05.004).
* **Loeiz Bourdic, Serge Salat, and Caroline Nowacki**. (2012) Assessing cities: a new system of cross-scale spatial indicators. *Building Research & Information*. 40(5):592–605. doi: [10.1080/09613218.2012.703488](https://doi.org/10.1080/09613218.2012.703488).
* **Zofie Cimburova**. (2017) Urban Morphology in Prague: Automatic Classification in GIS. [PhD thesis](https://dspace.cvut.cz/handle/10467/67903). *Czech Technical University, Prague*.
* **Jacob Dibble et al**. (2017) On the origin of spaces: Morphometric foundations of urban form evolution. *Environment and Planning B: Urban Analytics and City Science*. 46(4):707–730. doi: [10.1177/2399808317725075](https://doi.org/).
* **Alessandra Feliciotti**. (2018) Resilience and urban design: A systems approach to the study of resilience in urban form – Learning from the case of the Gorbals. PhD thesis, *University of Strathclyde, Glasgow*. doi: [10.48730/yrsa-7747](https://doi.org/10.48730/yrsa-7747).
* **Martin Fleischmann**. (2019) momepy: Urban Morphology Measuring Toolkit. *Journal of Open Source Software*. 4(43):1807. doi: [10.21105/joss.01807](https://doi.org/10.21105/joss.01807).
* **Martin Fleischmann**. (2021). martinfleis/clustergram: v0.6.0 (v0.6.0). *Zenodo*. doi: [10.5281/zenodo.5680799](https://doi.org/10.5281/zenodo.5680799).
* **Martin Fleischmann et al**. (2020) Morphological tessellation as a way of partitioning space: Improving consistency in urban morphology at the plot scale. *Computers, Environment and Urban Systems*. 80:101441. doi: [10.1016/j.compenvurbsys.2019.101441](https://doi.org/10.1016/j.compenvurbsys.2019.101441).
* **Martin Fleischmann et al**. (2021) Methodological Foundation of a Numerical Taxonomy of Urban Form. *Environment and Planning B: Urban Analytics and City Science*. 49(4):1283-1299 doi: [10.1177/23998083211059835](https://doi.org/10.1177/23998083211059835).
* **Jorge Gil et al**. (2012) On the Discovery of Urban Typologies: Data Mining the Multi-dimensional Character of Neighbourhoods. *[Urban Morphology](http://www.urbanform.org/online_public/2012_1.shtml)*, [16(1):27–40](http://www.urbanform.org/online_unlimited/pdf2012/201216_27.pdf).
* **Rachid Hamaina, Thomas Leduc, and Guillaume Moreau**. (2012) Towards Urban Fabrics Characterization Based on Buildings Footprints. In *Bridging the Geographic Information Sciences*, 2:327–346. Springer, Berlin, Heidelberg. doi: [10.1007/978-3-642-29063-3_18](https://doi.org/10.1007/978-3-642-29063-3_18).
* **Birgit Hausleitner and Meta Berghauser Pont**. (2017) Development of a configurational typology for micro-businesses integrating geometric and configurational variables. In *11th Space Syntax Symposium*. [66:1-14](https://pure.tudelft.nl/ws/portalfiles/portal/51462127/Paer_66_book_proceedings_28092017.pdf).
* **Txomin Hermosilla et al**. (2012) Assessing contextual descriptive features for plot-based classification of urban areas. *Landscape and Urban Planning*, 106(1):124–137. doi: [10.1016/j.landurbplan.2012.02.008](https://doi.org/10.1016/j.landurbplan.2012.02.008).
* **Kelsey Jordahl et al**. (2022). geopandas/geopandas: v0.11.1 (v0.11.1). *Zenodo*. doi: [10.5281/zenodo.6894736](https://doi.org/10.5281/zenodo.6894736).
* **Kevin McGarigal and Barbara J. Marks**. (1995) FRAGSTATS: Spatial Pattern Analysis Program for Quantifying Landscape Structure. *Gen. Tech. Rep. PNW-GTR-351. Portland, OR: U.S. Department of Agriculture, Forest Service, Pacific Northwest Research Station*. doi: [10.2737/PNW-GTR-351](https://doi.org/10.2737/PNW-GTR-351).
* **Sergio Porta, Paolo Crucitti, and Vito Latora**. (2006) The network analysis of urban streets: A primal approach. *Environment and Planning B: Planning and Design*. 33(5):705–725. doi: [10.1068/b32045](https://doi.org/10.1068/b32045).
* **Sergio J. Rey et al**. (2021) The PySAL Ecosystem: Philosophy and Implementation. *Geographical Analysis*, 54(3):467–487. doi: [10.1111/gean.12276](https://doi.org/10.1111/gean.12276).
* **Patrick M. Schirmer and Kay W. Axhausen**. (2015) A multiscale classification of urban morphology. *Journal of Transport and Land Use*. 9(1):101–130. doi: [10.5198/jtlu.2015.667](https://doi.org/10.5198/jtlu.2015.667).
* **Pratyush Tripathy et al**. (2020) An open-source tool to extract natural continuity and hierarchy of urban street networks. *Environment and Planning B: Urban Analytics and City Science*. 48(8):2188-2205, doi: [10.1177/2399808320967680](https://doi.org/10.1177/2399808320967680).
* **Sven Vanderhaegen and Frank Canters**. (2017) Mapping urban form and function at city block level using spatial metrics. *Landscape and Urban Planning*. 167:399–409. doi: [10.1016/j.landurbplan.2017.05.023](https://doi.org/10.1016/j.landurbplan.2017.05.023).

## Acknowledgement

The following acknowledgement applies to James D. Gaboardi (affiliation 4) only:

> This manuscript has been authored in part by UT-Battelle LLC under contract DE- AC05-00OR22725 with the US Department of Energy (DOE). The US government retains and the publisher, by accepting the article for publication, acknowledges that the US government retains a nonexclusive, paid-up, irrevocable worldwide license to publish or reproduce the published form of this manuscript, or allow others to do so, for US government purposes. DOE will provide public access to these results of federally sponsored research in accordance with the DOE Public Access Plan (http://energy.gov/downloads/doe-public-access-plan).